In [29]:
num_sample = 3000
num_burn = 200
sample_size = 800

In [30]:
from dgp import sample_network_chain, get_graph, sample_Y1, sample_Y2, agcEffect
import numpy as np

# 1. Simulate or load data
min_degree = 1
max_degree = 2
adj = get_graph(sample_size, min_degree, max_degree, seed=1)

tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
rho = np.array([[0,0.1,0.2],
                [0.1,0,0.1],
                [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
gamma = np.array([-2.00,0.50,-0.05,0.20,0.05,0.25,-0.08,0.30])     # shape (8,)
beta = np.array([-0.30,-0.60,-0.60,-0.20,-0.05,-0.10,-0.01,0.40,0.01,0.20])     # shape (10,)

Y_chain, A_chain, L_chain = sample_network_chain(adj, tau, rho, nu, gamma, beta, R=num_sample,
    burnin_R=num_burn, seed=0, sample_Y_func=sample_Y2, Atype=('gen', 0.7))

Y_chain = Y_chain[::3]
A_chain = A_chain[::3]
L_chain = L_chain[::3]

dir, dir2 = [], []
for i in range(Y_chain.shape[0]):
    Y = Y_chain[i]
    A = A_chain[i]
    L = L_chain[i]
    dir.append(np.mean(Y[A==1]) - np.mean(Y[A==0]))
    dir2.append(np.mean(A))

np.mean(dir), np.mean(dir2)

100%|██████████| 3200/3200 [00:07<00:00, 428.33it/s]


(-0.10474007661166362, 0.18246125000000002)

In [31]:
import networkx as nx
def compute_avg_effects_std_from_raw(psi_vec, adj_matrix, h=2):
    N = len(psi_vec)
    avg_effects = np.mean(psi_vec)
    
    # Centered residuals
    g = psi_vec - avg_effects
    
    # Build graph and compute pairwise distances
    G = nx.from_numpy_array(adj_matrix)
    dist = dict(nx.all_pairs_shortest_path_length(G, cutoff=h))
    
    # Network HAC estimator
    hac_var = 0.0
    for i in range(N):
        for j, dij in dist[i].items():
            weight = max(1 - dij / h, 0)  # Bartlett kernel
            hac_var += weight * g[i] * g[j]
    
    hac_var /= N
    se_hac = np.sqrt(hac_var)/np.sqrt(N)

    return avg_effects, se_hac

In [37]:
var = []
var_sim = []
est = []
idx = 1
for i in range(Y_chain.shape[0]):
    avg_effects, se_hac = compute_avg_effects_std_from_raw(L_chain[i,:,2],adj,h=3)
    var.append(se_hac**2*sample_size)
    var_sim.append(np.var(L_chain[i,:,2]))
    est.append(avg_effects)

var_true = np.var(est)
print(np.mean(var), np.mean(var_sim), var_true*sample_size)

0.24360134964583313 0.24385864375 0.240766195


In [38]:
k = 51
print(np.where(adj[k])[0])
for j in np.where(adj[k])[0]:
    print(np.corrcoef(L_chain[:,k,0], L_chain[:,j,0])[0,1])

ic = []
for j in np.where(adj[0]==0)[0]:
    ic.append(np.corrcoef(L_chain[:,k,0], L_chain[:,j,0])[0,1])
np.mean(ic)

[50 53]
0.012613046601269454
-0.04626766551011184


-0.0007506275090633911

In [40]:
L_chain.shape, Y_chain.shape

((1000, 800, 3), (1000, 800))